# Notebook Overview:

The intent of this notebook is to conduct initial Exploratory Data Analysis (EDA) on the Dataset provided: `SRDataEngineerChallenge_DATASET.csv`. This will allow for data insights to ensure a proper ETL pipeline can be produced for this dataset. 

### Load in Pandas:

In [1]:
import pandas as pd

### Read in dataset:

In [2]:
df = pd.read_csv("../SRDataEngineerChallenge_DATASET.csv")

### Explore Data:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1000 non-null   int64 
 1   first_name  1000 non-null   object
 2   last_name   1000 non-null   object
 3   email       1000 non-null   object
 4   gender      1000 non-null   object
 5   ip_address  1000 non-null   object
dtypes: int64(1), object(5)
memory usage: 47.0+ KB


Data set is small at only 47.0 KB. This is nice because it means we can work with it simply using Pandas and processing will not require any distributed compute. 

Additionally, data does not contain any null values. This means we do not have to determine how to handle nulls via deletion or imputation. 

Finally, we learn here that the data is largely text based. Outside of the initial index provided with the data, all of the data types are objects indicating non-numerical or string type values. 

In [ ]:
df.head()

Looking at the head of data a couple things stand out:

1. This is PII Data. Every column besides the `id` column can be considered a PII field. As a result, we must consider data privacy. As a result, a good transformation idea for our pipeline will be to scrub human PII via a hasing algorithm such as `MD5`. 
2. It appears as though gender values fall into discrete buckets. As a result, it may be best to map the data in that column to smaller values while still retaining the information they are holding. I.e. Male -> M
3. A relational data store appears to be the most obvious persistent storage solution for this data. The data is already structured with a well defined schema. 
4. If we intend to place this data into a relational data store, it will be important to consider which field(s) will make up the primary and foreign keys. A primary key must consist of a unique identifier for each column. Also, because the intent of this data is unkown at this time, it is difficult to make a firm decision on schema and keys as well. For now, let's assume that we want the Primary Key to be a column that we can share external to this datastore to link back to each individual's data. As a result, a hash of the `first_name`, `last_name`, `email`, and `ip_address` will ensure a single Primary Key for this table and will ensure that primary key can be used without worrying about displaying PII. 
5. ** For now, we will store all data fields until a better understanding of data privacy and regulation requirements for this data is understood.**

In [5]:
df['gender'].value_counts()

Genderfluid    140
Female         131
Genderqueer    130
Male           129
Agender        120
Bigender       119
Polygender     118
Non-binary     113
Name: gender, dtype: int64

We have a relatively equitabble distribution of discrete gender fields. This makes the gender field a good candidate for transformation via mapping to enable more efficient data storage in our relational database.

In [6]:
GENDER_MAP = {
    'Genderfluid':'GF',
    'Female':'F',
    'Genderqueer':'GQ',
    'Male':'M',
    'Agender':'A',
    'Bigender':'B',
    'Polygender':'P',
    'Non-binary':'NB'
}

One thing we want to note at this stage is that we may not know all of the potential gender field options at this time nor do we know how standard the inputs we receive will be. I.e. Will the first letter always be capitalized? Will non-binary always be hyponated? At this time, we will apply a default option if we don't receive an exact string match when we map our gender variables. But this is something to note and consider for future development for a more robust and scalable ETL Pipeline for generating more useful data. 

In [8]:
### Map gender values and provide 'UNKWN' string if input gender value is Unknown:
df['gender'] = df['gender'].apply(lambda x: GENDER_MAP.get(x, 'UNKWN'))

In [9]:
df['gender'].value_counts()

GF    140
F     131
GQ    130
M     129
A     120
B     119
P     118
NB    113
Name: gender, dtype: int64

Now let's execute the hashing of our PII to create our Primary Key.

In [ ]:
df.head()

In [25]:
import hashlib
df['concat'] = df['first_name'] + df['last_name'] + df['email'] + df['ip_address']
df['hash'] = df['concat'].apply(lambda x: hashlib.md5(x.encode()).hexdigest())
df.drop(columns = ['concat'], axis = 1, inplace = True)

In [27]:
df = df[['id','hash','first_name','last_name','email','gender','ip_address']]

In [ ]:
df.head()

### Conclusion:

For the Airflow portion of this solution, the Python ETL Pipeline will conduct the following actions:

1. Read a flat csv file into a Pandas DataFrame
2. Map gender fields and create a PII sensitve primary key
3. Ingest the dataframe into a PostgreSQL Database